<a href="https://colab.research.google.com/github/grillinr/evolutionary-computing/blob/main/final/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and seed for easier checking

In [50]:
# 0.66 is baseline for score
import random
import os
import argparse

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score, fbeta_score
from sklearn.model_selection import train_test_split

SEED = 5173
device = torch.device("cpu")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Define helper functions

In [59]:
def prepare_data(data, device):
    # Drop non-numeric and class columns
    data = data.dropna()
    X = data.drop(columns=["id", "record", "type"]).values.astype(np.float32)

    # Convert class to numeric value from 0-4
    y = data["type"].astype("category").cat.codes.values

    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y, dtype=torch.long).to(device)

    return X_tensor, y_tensor


def evaluate(model, device, data, criterion):
    model.eval()

    X, y = prepare_data(data, device)

    with torch.no_grad():
        logits = model(X)
        loss = criterion(logits, y)
        probs = torch.softmax(logits, dim=1).cpu().numpy()

    y_true = y.cpu().numpy()
    y_pred = probs.argmax(axis=1)

    acc = accuracy_score(y_true, y_pred)
    f_beta_m = fbeta_score(y_true, y_pred, average="macro", beta=2)

    return {
        "loss": loss.item(),
        "accuracy": acc,
        "f_beta_macro": f_beta_m,
    }

# Create Model Architecture (DNN)

In [52]:
class DNN(nn.Module):
    def __init__(self, input_size=32, hidden=(32, 16, 8), num_classes=5, dropout_rate=0.5):
        super().__init__()
        layers = []
        input_dim = input_size

        for h in hidden:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU(inplace=True))
            layers.append(nn.Dropout(dropout_rate))
            input_dim = h

        layers.append(nn.Linear(input_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# Main Training loop

In [60]:
# Load data
dataset = pd.read_csv("/content/train.csv")
train_dataset, test_dataset = train_test_split(dataset, train_size=0.7, random_state=SEED)
# test_dataset = pd.read_csv("test.csv")

In [63]:
# Configuration
lr = 1e-3
epochs = 100
hidden = (32, 16, 8)
dropout_rate = 0.5
patience = 10

# Create model
model = DNN(hidden=hidden, dropout_rate=dropout_rate).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [61]:
# Prepare tensors
X_train, y_train = prepare_data(train_dataset, device)

tensor(0)

In [65]:
# Training loop with early stopping
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    train_loss = loss.item()

    train_metrics = evaluate(model, device, train_dataset, criterion)
    val_metrics = evaluate(model, device, test_dataset, criterion)

    print(
        f"Epoch {epoch}/{epochs} | "
        f"train_loss={train_loss:.4f} train_acc={train_metrics['accuracy']:.4f} "
        f"train_f1={train_metrics['f_beta_macro']:.4f} | "
        f"val_loss={val_metrics['loss']:.4f} val_acc={val_metrics['accuracy']:.4f} "
        f"val_f1={val_metrics['f_beta_macro']:.4f}"
    )

    # Early stopping check
    if val_metrics['loss'] < best_val_loss:
        best_val_loss = val_metrics['loss']
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

Epoch 1/100 | train_loss=4.3106 train_acc=0.8344 train_f1=0.2437 | val_loss=1.2239 val_acc=0.8346 val_f1=0.2431
Epoch 2/100 | train_loss=3.9643 train_acc=0.8801 train_f1=0.2308 | val_loss=1.1346 val_acc=0.8795 val_f1=0.2309
Epoch 3/100 | train_loss=3.6687 train_acc=0.8988 train_f1=0.2116 | val_loss=1.0714 val_acc=0.8989 val_f1=0.2147
Epoch 4/100 | train_loss=3.3967 train_acc=0.9041 train_f1=0.2097 | val_loss=1.0262 val_acc=0.9042 val_f1=0.2129
Epoch 5/100 | train_loss=3.1573 train_acc=0.9048 train_f1=0.2036 | val_loss=0.9902 val_acc=0.9046 val_f1=0.2076
Epoch 6/100 | train_loss=2.9443 train_acc=0.9050 train_f1=0.2014 | val_loss=0.9597 val_acc=0.9047 val_f1=0.1991
Epoch 7/100 | train_loss=2.7721 train_acc=0.9049 train_f1=0.2007 | val_loss=0.9326 val_acc=0.9046 val_f1=0.1981
Epoch 8/100 | train_loss=2.6036 train_acc=0.9048 train_f1=0.2002 | val_loss=0.9075 val_acc=0.9047 val_f1=0.1977
Epoch 9/100 | train_loss=2.4583 train_acc=0.9048 train_f1=0.1971 | val_loss=0.8852 val_acc=0.9046 val_f1

KeyboardInterrupt: 

# Test output on split data to avoid submitting

In [ ]:
X_test, y_test = prepare_data(test_dataset, device)

# Get predictions
with torch.no_grad():
    logits = model(X_test)
    probs = torch.softmax(logits, dim=1).cpu().numpy()
    predictions = probs.argmax(axis=1)

y_true = y_test.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(y_true, predictions)
f_beta = fbeta_score(y_true, predictions, average="macro", beta=2)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F-Beta (macro): {f_beta:.4f}")